In [9]:
import numpy as np
import os
import pandas as pd
import seaborn as sns
import matplotlib.pyplot as plt
from IPython.display import Image
from transvae import trans_models
from transvae.transformer_models import TransVAE
from transvae.rnn_models import RNN, RNNAttn
from transvae.wae_models import WAE
from transvae.aae_models import AAE
from transvae.tvae_util import *
from transvae import analysis
import glob
import re

from sklearn.decomposition import PCA
from sklearn.manifold import Isomap
from sklearn import metrics
from sklearn.manifold import trustworthiness
import numpy as np
import matplotlib.pyplot as plt
from matplotlib.ticker import NullFormatter
import plotly.express as px

import coranking #coranking.readthedocs.io
from coranking.metrics import trustworthiness, continuity, LCMC
from transvae.snc import SNC #github.com/hj-n/steadiness-cohesiveness

def loss_plots(loss_src):
    tot_loss = analysis.plot_loss_by_type(src,loss_types=['tot_loss'])
    plt.savefig(save_dir+'tot_loss.png')
    recon_loss = analysis.plot_loss_by_type(src,loss_types=['recon_loss'])
    plt.savefig(save_dir+'recon_loss.png')
    kld_loss = analysis.plot_loss_by_type(src,loss_types=['kld_loss'])
    plt.savefig(save_dir+'kld_loss.png')
    prob_bce_loss = analysis.plot_loss_by_type(src,loss_types=['prop_bce_loss'])
    plt.savefig(save_dir+'prob_bce_loss.png')
    if 'aae' in src:
        disc_loss = analysis.plot_loss_by_type(src,loss_types=['disc_loss'])
        plt.savefig(save_dir+'disc_loss.png')
    if 'wae' in src:
        mmd_loss = analysis.plot_loss_by_type(src,loss_types=['mmd_loss'])
        plt.savefig(save_dir+'mmd_loss.png')
    plt.close('all')
    
def load_reconstructions(data,data_1D,latent_size, load_src, true_props=None,subset=None):
    
    recon_src = load_src+model.name+"_"+re.split('(\d{2,3})',latent_size[0])[0]+"_"+re.split('(\d{2,3})',latent_size[0])[1]+"//saved_info.csv"
    recon_df = pd.read_csv(recon_src)
    reconstructed_seq = recon_df['reconstructions'].to_list()[:num_sequences]
    props = torch.Tensor(recon_df['predicted properties'][:num_sequences])
    true_props_data = pd.read_csv(true_props).to_numpy()
    true_props = true_props_data[0:num_sequences,0]
    
    if subset:
        testing = pd.read_csv(subset).to_numpy()
        test_idx_list = [np.where(data==testing[idx][0]) for idx in range(len(testing))]


        batch_recon_len = len(reconstructed_seq)
        reconstructed_seq = [reconstructed_seq[test_idx_list[i][0][0]] for i in range(len(test_idx_list)) if test_idx_list[i][0][0]<batch_recon_len]
        data_1D= [data_1D[test_idx_list[i][0][0]] for i in range(len(test_idx_list)) if test_idx_list[i][0][0]<batch_recon_len]
        props = [props[test_idx_list[i][0][0]] for i in range(len(test_idx_list)) if test_idx_list[i][0][0]<batch_recon_len]
        props=torch.Tensor(props)
        data = testing[:][0]
        true_props_data = pd.read_csv(true_props).to_numpy()
        true_props = true_props_data[0:num_sequences,0]
        true_props= [true_props[test_idx_list[i][0][0]] for i in range(len(test_idx_list)) if test_idx_list[i][0][0]<batch_recon_len]

    return data, data_1D, true_props, props, reconstructed_seq

########################################################################################
gpu = True

num_sequences = 500#_000
batch_size = 200 #setting for reconstruction
example_data = 'slurm_analyses//data//sunistar//peptide_train.txt'
save_dir_loc = 'slurm_analyses' #folder in which to save outpts
save_dir_name = 'train' #appended to identify data: train|test|other|etc...

reconstruct=True #True:reconstruct data here; False:load reconstructions from file
recon_src = "checkpointz//analyses_ckpts//" #directory in which all reconstructions are stored
true_prop_src = "slurm_analyses//data//sunistar//function_train.txt" #if property predictor load the true labels
subset_src = "" #(optional) this file should have the true sequences for a subset of the "example data" above

ckpt_list = glob.glob(""+"temp_ckpt//**//*.ckpt", recursive = True) #grab all checkpoint
print('current working directory: ',os.getcwd())


for i in range(len(ckpt_list)):
    
    #search the current directory for the model name and load that model
    model_dic = {'trans':'TransVAE','aae':'AAE','rnn':'RNN','rnnattn':'RNNAttn','wae':'WAE'}
    model_src = ckpt_list[i]
    print('working on: ',model_src,'\n')
    model_name = list(filter(None,[key for key in model_dic.keys() if key in model_src.split('\\')[-1]]))
    model = locals()[model_dic[model_name[0]]](load_fn=model_src) #use locals to call model specific constructor
    
    #create save directory for the current model according to latent space size
    latent_size = re.findall('(latent[\d]{2,3})', model_src)
    save_dir= save_dir_loc+model.name+latent_size[0]+save_dir_name
    if not os.path.exists(save_dir):os.mkdir(save_dir) 
    save_dir= save_dir+"//" 
    save_df = pd.DataFrame() #this will hold the number variables and save to CSV
    
    #load the true labels
    data = pd.read_csv(example_data).to_numpy() 
    data_1D = data[:num_sequences,0] #gets rid of extra dimension
    true_props_data = pd.read_csv(true_prop_src).to_numpy()
    true_props = true_props_data[0:num_sequences,0]

    
    #get the log.txt file from the ckpt and model name then plot loss curves
    
    loss_src = '_'.join( ("log",model_src.split('\\')[-1].split('_')[1],model_src.split('\\')[-1].split('_')[2][:-4]+"txt") )
    src= '\\'.join([str(i) for i in model_src.split('\\')[:-1]])+"\\"+loss_src
    print(loss_src, src)
    loss_plots(src)
    
    #set the batch size and reconstruct the data
    model.params['BATCH_SIZE'] = batch_size
    if reconstruct:
        reconstructed_seq, props = model.reconstruct(data[:num_sequences], log=False, return_mems=False)
    else:
        data, data_1D, true_props, props, reconstructed_seq = load_reconstructions(data, data_1D,latent_size,
                                                                                   load_src=recon_src,
                                                                                   true_props=true_prop_src)
    if gpu:torch.cuda.empty_cache() #free allocated CUDA memory
    
    #save the metrics to the dataframe
    save_df['reconstructions'] = reconstructed_seq #placing the saves on a line separate from the ops allows for editing
    save_df['predicted properties'] = [prop.item() for prop in props[:len(reconstructed_seq)]]
    prop_acc, prop_conf, MCC=calc_property_accuracies(props[:len(reconstructed_seq)],true_props[:len(reconstructed_seq)], MCC=True)
    save_df['property prediction accuracy'] = prop_acc
    save_df['property prediction confidence'] = prop_conf
    save_df['MCC'] = MCC
    

#   First we tokenize the input and reconstructed smiles
    input_sequences = []
    for seq in data_1D:
        input_sequences.append(peptide_tokenizer(seq))
    output_sequences = []
    for seq in reconstructed_seq:
        output_sequences.append(peptide_tokenizer(seq))
    
    seq_accs, tok_accs, pos_accs, seq_conf, tok_conf, pos_conf = calc_reconstruction_accuracies(input_sequences, output_sequences)
    save_df['sequence accuracy'] = seq_accs
    save_df['sequence confidence'] = seq_conf
    save_df['token accuracy'] = tok_accs
    save_df['token confidence'] = tok_conf
    save_df = pd.concat([pd.DataFrame({'position_accs':pos_accs,'position_confidence':pos_conf }), save_df], axis=1)
    
    ##moving into memory and entropy
    if model.model_type =='aae':
        mus, _, _ = model.calc_mems(data[:], log=False, save=False) 
    elif model.model_type == 'wae':
        mus, _, _ = model.calc_mems(data[:], log=False, save=False) 
    else:
        mems, mus, logvars = model.calc_mems(data[:1_000], log=False, save=False) #subset size 1200*35=42000 would be ok


    ##calculate the entropies
    vae_entropy_mus = calc_entropy(mus)
    save_df = pd.concat([save_df,pd.DataFrame({'mu_entropies':vae_entropy_mus})], axis=1)
    if model.model_type != 'wae' and model.model_type!= 'aae': #these don't have a variational type bottleneck
        vae_entropy_mems  = calc_entropy(mems)
        save_df = pd.concat([save_df,pd.DataFrame({'mem_entropies':vae_entropy_mems})], axis=1)
        vae_entropy_logvars = calc_entropy(logvars)
        save_df = pd.concat([save_df,pd.DataFrame({'logvar_entropies':vae_entropy_logvars})], axis=1)
    


    #create random index and re-index ordered memory list creating n random sub-lists (ideally resulting in IID random lists)
    random_idx = np.random.permutation(np.arange(stop=mus.shape[0]))
    mus[:] = mus[random_idx]
    data = data[random_idx]

    #define the subset of the data to sample for PCA and silhouette/cluster metrics
    subsample_start=0
    subsample_length=mus.shape[0]

    #(for length based coloring): record all peptide lengths iterating through input
    pep_lengths = []
    for idx, pep in enumerate(data[subsample_start:(subsample_start+subsample_length)]):
        pep_lengths.append( len(pep[0]) )   
    #(for function based coloring): pull function from csv with peptide functions

    s_to_f =pd.read_csv(true_prop_src)    
    function = s_to_f['peptides'][subsample_start:(subsample_start+subsample_length)]
    function = function[random_idx] #account for random permutation

    pca = PCA(n_components=2)
    pca_batch =pca.fit_transform(X=mus[:])

    fig = px.scatter(pca_batch,color= pep_lengths ,opacity=0.7)
    fig.update_traces(marker=dict(size=3))
    fig.write_image(save_dir+'pca_length.png', width=1920, height=1080)

    fig = px.scatter_matrix(pca_batch, color= [str(itm) for itm in function], opacity=0.7)
    fig.update_traces(marker=dict(size=3))
    fig.write_image(save_dir+'pca_function.png', width=1920, height=1080)

    #create n subsamples and calculate silhouette score for each
    latent_mem_func_subsamples = []
    pca_func_subsamples = []
    n=250
    for s in range(n):
        s_len = len(mus)//n #sample lengths
        mem_func_sil = metrics.silhouette_score(mus[s_len*s:s_len*(s+1)], function[s_len*s:s_len*(s+1)], metric='euclidean')
        latent_mem_func_subsamples.append(mem_func_sil)
        XY = [i for i in zip(pca_batch[s_len*s:s_len*(s+1),0], pca_batch[s_len*s:s_len*(s+1),1])]
        pca_func_sil = metrics.silhouette_score(XY, function[s_len*s:s_len*(s+1)], metric='euclidean')
        pca_func_subsamples.append(pca_func_sil)
    save_df = pd.concat([save_df,pd.DataFrame({'latent_mem_func_silhouette':latent_mem_func_subsamples})], axis=1)
    save_df = pd.concat([save_df,pd.DataFrame({'pca_func_silhouette':pca_func_subsamples})], axis=1)

    
    save_df.to_csv(save_dir+"saved_info.csv", index=False)

current working directory:  C:\Users\s_renaud\Documents\GitHub\MSCSAM_TBD\main_model
working on:  temp_ckpt\rnn_latent128\300_rnn-128_peptide.ckpt 

log_rnn-128_peptide.txt temp_ckpt\rnn_latent128\log_rnn-128_peptide.txt
rnn-128_peptide
cuda
decoding sequences of max length  125 current position:  0
decoding sequences of max length  125 current position:  10
decoding sequences of max length  125 current position:  20
decoding sequences of max length  125 current position:  30
decoding sequences of max length  125 current position:  40
decoding sequences of max length  125 current position:  50


KeyboardInterrupt: 

<H4>Since Compute Canada does not do the dimensionality reduction metrics we need to do them below

In [1]:
import coranking #coranking.readthedocs.io
from coranking.metrics import trustworthiness, continuity, LCMC
from transvae.snc import SNC #github.com/hj-n/steadiness-cohesiveness

import numpy as np
import os
import pandas as pd
import seaborn as sns
from sklearn.decomposition import PCA
import matplotlib.pyplot as plt
from IPython.display import Image
from transvae import trans_models
from transvae.transformer_models import TransVAE
from transvae.rnn_models import RNN, RNNAttn
from transvae.wae_models import WAE
from transvae.aae_models import AAE
from transvae.tvae_util import *
from transvae import analysis
import glob
import re


gpu = True

example_data = 'data\\peptides\\datasets\\uniprot_v2\\peptide_train.txt'
test_train='train'
ckpt_list = glob.glob(""+"checkpointz\\to_slurm//**//*.ckpt", recursive = True) #grab all checkpoints
analyses_list = glob.glob("model_analyses\\train//**/*.csv", recursive=True) #grab all analyses
print('current working directory: ',os.getcwd())

for i in range(len(ckpt_list)):
    
    #search the current directory for the model name and load that model
    model_dic = {'trans':'TransVAE','aae':'AAE','rnnattn':'RNNAttn','rnn':'RNN','wae':'WAE'}
    model_src = ckpt_list[i]
    print('working on: ',model_src,'\n')
    model_name = list(filter(None,[key for key in model_dic.keys() if key in model_src.split('//')[-1]]))
    model = locals()[model_dic[model_name[0]]](load_fn=model_src) #use locals to call model specific constructor
    
    #load the analysis file corresponding to the model from the CC outputs
    for idx in range(len(analyses_list)):
        if analyses_list[idx].split("\\")[-2].find(model_src.split("\\")[-2].split("_")[0]) != -1 and analyses_list[idx].split("\\")[-2].find(model_src.split("\\")[-2].split("_")[1]) != -1:
            if analyses_list[idx].find("rnnattn")  != -1 and model_src.find("rnnattn") == -1: continue
            save_dir = analyses_list[idx]
            cur_analysis = pd.read_csv(save_dir)
    print("analysis: ",save_dir, "checkpoint: ",model_src)
    save_df = cur_analysis #this will hold the number variables and save to CSV
    
    #load the true labels
    data = pd.read_csv(example_data).to_numpy() 
    data_1D = data[:,0] #gets rid of extra dimension
    
    #moving into memory and entropy
    if model.model_type =='aae':
        mus, _, _ = model.calc_mems(data[:60_000], log=False, save=False) 
    elif model.model_type == 'wae':
        mus, _, _ = model.calc_mems(data[:60_000], log=False, save=False) 
    else:
        mems, mus, logvars = model.calc_mems(data[:60_000], log=False, save=False) #subset size 1200*35=42000 would be ok

    #create random index and re-index ordered memory list creating n random sub-lists (ideally resulting in IID random lists)
    random_idx = np.random.permutation(np.arange(stop=mus.shape[0]))
    mus[:] = mus[random_idx]
    data = data[random_idx]
    
    #need to perform PCA to be able to compare dimensionality reduction quality
    pca = PCA(n_components=2)
    pca_batch =pca.fit_transform(X=mus[:])
    
    #now ready to calculation dimensionality reduction accuracy with metrics
    trust_subsamples = []
    cont_subsamples = []
    lcmc_subsamples = []
    steadiness_subsamples = []
    cohesiveness_subsamples = []
    if 'test' in test_train: #different number of bootsraps for train vs test
        n=15
    else:
        n=60
    parameter = { "k": 50,"alpha": 0.1 } #for steadiness and cohesiveness
    for s in range(n):
        s_len = len(mus)//n
        Q = coranking.coranking_matrix(mus[s_len*s:s_len*(s+1)], pca_batch[s_len*s:s_len*(s+1)])
        trust_subsamples.append( np.mean(trustworthiness(Q, min_k=1, max_k=50)) )
        cont_subsamples.append( np.mean(continuity(Q, min_k=1, max_k=50)) )
        lcmc_subsamples.append( np.mean(LCMC(Q, min_k=1, max_k=50)) )
        print(s,trust_subsamples[s],cont_subsamples[s],lcmc_subsamples[s])

        metrics = SNC(raw=mus[s_len*s:s_len*(s+1)], emb=pca_batch[s_len*s:s_len*(s+1)], iteration=300, dist_parameter=parameter)
        metrics.fit() #solve for steadiness and cohesiveness
        steadiness_subsamples.append(metrics.steadiness())
        cohesiveness_subsamples.append(metrics.cohesiveness())
        print(metrics.steadiness(),metrics.cohesiveness())
        Q=0 #trying to free RAM
        metrics=0
        torch.cuda.empty_cache() #free allocated CUDA memory

    save_df = pd.concat([save_df,pd.DataFrame({'latent_to_PCA_trustworthiness':trust_subsamples})], axis=1)
    save_df = pd.concat([save_df,pd.DataFrame({'latent_to_PCA_continuity':cont_subsamples})], axis=1)
    save_df = pd.concat([save_df,pd.DataFrame({'latent_to_PCA_lcmc':lcmc_subsamples})], axis=1)
    save_df = pd.concat([save_df,pd.DataFrame({'latent_to_PCA_steadiness':steadiness_subsamples})], axis=1)
    save_df = pd.concat([save_df,pd.DataFrame({'latent_to_PCA_cohesiveness':cohesiveness_subsamples})], axis=1)  
    
    save_df.to_csv(save_dir, index=False)

current working directory:  C:\Users\s_renaud\Documents\GitHub\MSCSAM_TBD\main_model
working on:  checkpointz\to_slurm\rnnattn_latent128\300_rnnattn-128_peptide.ckpt 

analysis:  model_analyses\train\rnnattn-128_peptide_latent128_train\saved_info.csv checkpoint:  checkpointz\to_slurm\rnnattn_latent128\300_rnnattn-128_peptide.ckpt
rnnattn-128_peptide
0 0.6364532925810359 0.7671578504145421 0.048049687543560246
0.693426613960434 0.4642010741425193
1 0.6429021051520151 0.7698001259452246 0.05026090656692429
0.6814258682107179 0.4384161055892021
2 0.6371543939858083 0.7621945324350033 0.04287160411737432
0.6894073103770594 0.4303143792561397
3 0.6423069199657505 0.7643610599585914 0.04637994462602432
0.6889322599421552 0.43706911185346675
4 0.643468631107008 0.7651989629120498 0.04677060821780094
0.6816201246842429 0.44863847053714667
5 0.6344647165286765 0.7630344157383262 0.046307439650174614
0.6796554784865723 0.4531843915586711
6 0.638226509053012 0.7711833551419967 0.04832766522873867

C:\Users\s_renaud\Documents\GitHub\MSCSAM_TBD\main_model\transvae\trans_models.py:54: UserWarning: To copy construct from a tensor, it is recommended to use sourceTensor.clone().detach() or sourceTensor.clone().detach().requires_grad_(True), rather than torch.tensor(sourceTensor).
  self.params['CHAR_WEIGHTS'] = torch.tensor(self.params['CHAR_WEIGHTS'], dtype=torch.float)


rnnattn-128_peptide
0 0.8384522230030854 0.9235337355544967 0.15880356656005784
0.832481072101788 0.6974085802290942
1 0.8417565074869789 0.9230217551575141 0.15281305050312238
0.8323442756632685 0.6979836797755965
2 0.843503919424516 0.9230839383758853 0.1593008181870164
0.8298654189297223 0.7191463520479036
3 0.8416483789764045 0.9242148418609422 0.16388751840077198
0.8334472246161361 0.6931978439832015
4 0.8447562890740725 0.9254413002691716 0.1607512256527463
0.8235639190069443 0.7142438140334104
5 0.838393161102691 0.921605915776146 0.15352618756577555
0.8210963540463259 0.7106035648829889
6 0.8429855939290746 0.9238019951300753 0.15895373485704045
0.8329007768327794 0.6955439241549366
7 0.8364346972503807 0.922809033715651 0.15769221687705273
0.8429978412159806 0.7129902278779192
8 0.8470419451441928 0.9247824928963421 0.15995336373275584
0.8322600832029171 0.7168976571833349
9 0.841032968416944 0.9234040460156965 0.15658999993394287
0.8299841428568207 0.7051222966338269
10 0.840

C:\Users\s_renaud\Documents\GitHub\MSCSAM_TBD\main_model\transvae\trans_models.py:54: UserWarning: To copy construct from a tensor, it is recommended to use sourceTensor.clone().detach() or sourceTensor.clone().detach().requires_grad_(True), rather than torch.tensor(sourceTensor).
  self.params['CHAR_WEIGHTS'] = torch.tensor(self.params['CHAR_WEIGHTS'], dtype=torch.float)


rnnattn-128_peptide
0 0.6758351569758408 0.7869056967822203 0.07011256083971461
0.6755320177679631 0.587867033698456
1 0.6753868326573226 0.7860161560736812 0.06967341767500244
0.6770734691757805 0.5570332045184152
2 0.6830995123899389 0.7909959869920774 0.07380796972181831
0.6768028492138356 0.5722166976689214
3 0.6742777512017197 0.783188369366096 0.06862435661495798
0.6686786495403296 0.5798370090417484
4 0.6811731453123369 0.7913896601610445 0.07370719144813473
0.6791509002973708 0.5507305059583922
5 0.6855068212133167 0.7907590211655326 0.07252069486010339
0.6734582524568355 0.5890321761493424
6 0.6758133890639908 0.7884472390123578 0.07043961742006029
0.6819528989800033 0.5798200041222492
7 0.6805628797866259 0.792284003450435 0.073385974461423
0.677337069684977 0.5690963656108452
8 0.6823006703547776 0.794138375108415 0.07568706297015772
0.6763716608951797 0.559600398467311
9 0.6788461127970735 0.796068835186766 0.07232721155185305
0.6859989512929323 0.5502616765813392
10 0.6743

21 0.7231452707688631 0.8008994169329635 0.07219270508577387
0.7094068132341336 0.620197655732515
22 0.7249411550298046 0.801411209766528 0.07059453956733039
0.7014288208653037 0.6363140884176435
23 0.7231157923913228 0.8006637952453343 0.07225086325447924
0.6994610200552884 0.6419278285788512
24 0.7143385871991591 0.7911855007095875 0.07090111731961897
0.7120729931120775 0.6202146665944224
25 0.7179405507420868 0.7920531148437532 0.07019550976923852
0.7086439355896181 0.6157696747226269
26 0.7100776709072179 0.7855327412812253 0.07305985064229865
0.7100023213046724 0.6282561739545571
27 0.7182134197749116 0.7984754155152832 0.07158769820514907
0.7047074209846125 0.6471617442846525
28 0.719461633353317 0.7939579305014665 0.07336575764158718
0.6937691031740919 0.6406609345466325
29 0.7180728564967899 0.7929192849569042 0.07367413676182694
0.7179247520104536 0.6308781161238255
30 0.7120663430836114 0.7910903332664088 0.06896750415841021
0.6879362536416871 0.6563941828425452
31 0.72242759

C:\Users\s_renaud\Documents\GitHub\MSCSAM_TBD\main_model\transvae\trans_models.py:54: UserWarning: To copy construct from a tensor, it is recommended to use sourceTensor.clone().detach() or sourceTensor.clone().detach().requires_grad_(True), rather than torch.tensor(sourceTensor).
  self.params['CHAR_WEIGHTS'] = torch.tensor(self.params['CHAR_WEIGHTS'], dtype=torch.float)


rnn-128_peptide
0 0.6981357467923908 0.7848349001668847 0.06771276185508952
0.6298446422878464 0.7022324704158764
1 0.6917548319333615 0.7770062546785074 0.06662956547028716
0.6224804292355276 0.708226759116535
2 0.69061997512593 0.7779346448867838 0.0657289867221412
0.6225011832436343 0.70343565095939
3 0.695386182327458 0.7847403008445052 0.07029238496602844
0.6293328679023815 0.7204226996495195
4 0.6787471851046943 0.7688940708251428 0.06356574007515371
0.605346536175954 0.7277901052224109
5 0.6919817001087152 0.7820036563441894 0.06551060126330328
0.619586574702642 0.7230833736836275
6 0.6936411962770541 0.780159176643747 0.06824935859054569
0.6254755891139726 0.7263469259177096
7 0.6919908821461939 0.7781421800105283 0.06679116420221462
0.6134675292433474 0.7089883998916859
8 0.6836861927143582 0.7758615318483801 0.06144279270276254
0.6227164760222259 0.7150821286483717
9 0.6927336697321284 0.7820538988141534 0.068687337356924
0.6508417750819124 0.6954039657119248
10 0.68321978749

C:\Users\s_renaud\Documents\GitHub\MSCSAM_TBD\main_model\transvae\trans_models.py:54: UserWarning: To copy construct from a tensor, it is recommended to use sourceTensor.clone().detach() or sourceTensor.clone().detach().requires_grad_(True), rather than torch.tensor(sourceTensor).
  self.params['CHAR_WEIGHTS'] = torch.tensor(self.params['CHAR_WEIGHTS'], dtype=torch.float)


rnn-128_peptide
0 0.7394327742096667 0.8202182901817343 0.08386264346223858
0.7196669189916141 0.6323554017430095
1 0.7244201558153726 0.8079786622972973 0.07287773650989368
0.7173854359005294 0.6201927247321511
2 0.7332780788637941 0.815682860088684 0.07635739668010741
0.714196147186135 0.640002546286997
3 0.7291025981068762 0.8127009417946975 0.07489854336791724
0.7078419642152264 0.6202355081956057
4 0.737595810939327 0.8216940385961413 0.07416593792958613
0.7219723796784719 0.6373949529254983
5 0.7311428494947155 0.8146513624232058 0.07465035286472758
0.7150063753413735 0.6391460068738888
6 0.7339686251176788 0.8147857395965904 0.07934849193877887
0.7231982108833711 0.6248332928345535
7 0.732371277409782 0.8192427476449344 0.0814260283127909
0.7412001313096535 0.5872614114160537
8 0.7358574121874946 0.8139223471491355 0.07535855955242082
0.7326197788052962 0.6080793974885925
9 0.7411789136413814 0.8227103745607501 0.08211789378537653
0.7265306823405464 0.619013475720646
10 0.720610